In [ ]:
!nvidia-smi

Thu May  6 05:46:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    34W / 250W |  11311MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 99% 1.02G/1.04G [00:13<00:00, 75.5MB/s]
100% 1.04G/1.04G [00:13<00:00, 80.5MB/s]


In [ ]:
!rm -r Image-Captioning/

In [ ]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 354 (delta 206), reused 232 (delta 86), pack-reused 0
Receiving objects: 100% (354/354), 30.85 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (206/206), done.


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 7  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [ ]:
DATA_NAME = 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/data.json'
IMGS_PATH = 'flickr8k/images/'

In [ ]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 40000/40000 [00:00<00:00, 286477.59it/s]


In [ ]:
len(vocab)

5089

In [ ]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

In [ ]:
t_params

{'batch_size': 256,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 7,
 'fine_tune_encoder': False,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd533f4e10>,
 'workers': 2}

In [ ]:
fit(t_params=t_params, m_params=m_params)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/118]	Batch Time 4.755 (4.755)	Data Load Time 3.266 (3.266)	Loss 9.4578 (9.4578)	Top-5 Accuracy 0.195 (0.195)


In [ ]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop.pth.tar /content/drive/MyDrive/weights/

In [ ]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop.pth.tar'
epochs = 15

t_params = {
    'data_name': DATA_NAME+'_finetune',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr*.8,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune',
 'decoder_lr': 0.00032,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 15,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd545bc0d0>,
 'workers': 2}

In [ ]:
fit(t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 5
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [5][0/469]	Batch Time 1.736 (1.736)	Data Load Time 0.902 (0.902)	Loss 3.8268 (3.8268)	Top-5 Accuracy 68.293 (68.293)
Epoch: [5][100/469]	Batch Time 0.704 (0.720)	Data Load Time 0.000 (0.010)	Loss 4.1850 (4.0490)	Top-5 Accuracy 62.500 (63.931)
Epoch: [5][200/469]	Batch Time 0.705 (0.715)	Data Load Time 0.000 (0.006)	Loss 4.0673 (4.0145)	Top-5 Accuracy 64.947 (64.577)
Epoch: [5][300/469]	Batch Time 0.692 (0.713)	Data Load Time 0.000 (0.004)	Loss 3.8470 (3.9893)	Top-5 Accuracy 67.878 (64.951)
Epoch: [5][400/469]	Batch Time 0.715 (0.712)	Data Load Time 0.000 (0.004)	Loss 3.8155 (3.9740)	Top-5 Accuracy 68.421 (65.176)
---

KeyboardInterrupt: ignored

In [ ]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar /content/drive/MyDrive/weights/

In [ ]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar')

Loaded Checkpoint!!
Last Epoch: 12
Best Bleu-4: 0.15974322033502095


In [ ]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar'
epochs = 15

t_params = {
    'data_name': DATA_NAME+'_finetune_new_lr',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr*.8,
    'encoder_lr': encoder_lr*.8,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune_new_lr',
 'decoder_lr': 0.00032,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 8e-05,
 'epochs': 15,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd545bc0d0>,
 'workers': 2}

In [ ]:
fit(t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 13
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [13][0/469]	Batch Time 1.702 (1.702)	Data Load Time 0.922 (0.922)	Loss 2.9290 (2.9290)	Top-5 Accuracy 79.778 (79.778)
Epoch: [13][100/469]	Batch Time 0.711 (0.723)	Data Load Time 0.000 (0.011)	Loss 2.7991 (2.7624)	Top-5 Accuracy 81.139 (82.264)
Epoch: [13][200/469]	Batch Time 0.719 (0.717)	Data Load Time 0.000 (0.006)	Loss 2.9192 (2.7909)	Top-5 Accuracy 80.526 (81.872)
Epoch: [13][300/469]	Batch Time 0.743 (0.715)	Data Load Time 0.012 (0.005)	Loss 2.8699 (2.8093)	Top-5 Accuracy 80.472 (81.621)
Epoch: [13][400/469]	Batch Time 0.711 (0.713)	Data Load Time 0.000 (0.004)	Loss 2.8068 (2.8186)	Top-5 Accuracy 82.283 (81.48

In [ ]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam_finetune.pth.tar')

Loaded Checkpoint!!
Last Epoch: 5
Best Bleu-4: 0.15255731663785566


In [ ]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_adam.pth.tar')

Loaded Checkpoint!!
Last Epoch: 4
Best Bleu-4: 0.1466164024634058
